# Install Dependencies

In [11]:
%pip install tensorflow
%pip install tensorflow-gpu
%pip install tensorflow-hub
%pip install opencv-python
%pip install matplotlib
%pip install sklearn

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\lance\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\lance\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


...

You should consider upgrading via the 'c:\Users\lance\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\lance\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\lance\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


# Import Dependencies

In [12]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
from matplotlib import pyplot as plt
import numpy as np
import os

In [13]:
# Optional if you are using a GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

[]


# Load Movenet Model

In [14]:
model = hub.load('https://tfhub.dev/google/movenet/multipose/lightning/1')
movenet = model.signatures['serving_default']

# Make Detections

In [15]:
# Fucntion that draws points of the joints on the screen
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 6, (0,255,0), -1)

In [16]:
# object that mapes the connections between each point drawn on the screen
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [17]:
# Function that draws the edges connecting the points drwan by draw_keypoints fucntion
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 4)

In [18]:
# Function to loop through each person detected and render with maximum number 6 persons
def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold):
    for person in keypoints_with_scores:
        draw_connections(frame, person, edges, confidence_threshold)
        draw_keypoints(frame, person, confidence_threshold)

# Open Camera and Render Skeleton

In [19]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    
    # Resize image
    img = frame.copy()
    img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0),192,256)
    input_img = tf.cast(img, dtype=tf.int32)
    
    # Detection section
    results = movenet(input_img)
    keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
    
    # Render keypoints 
    loop_through_people(frame, keypoints_with_scores, EDGES, 0.35)
    
    cv2.imshow('Human Gait Identification', frame)
    
    if cv2.waitKey(10) & 0xFF== 27:
        break
cap.release()
cv2.destroyAllWindows()

# Folder Setup for Data Collection

In [20]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('Custom_Gait_Dataset') 

# Persons that we try to detect
persons = np.array(['Lance','Seif'])

#Very person with 4 different view
views = np.array(['Front','Back','Left','Right'])

# 10 videos worth of data
no_sequences = 20

# Videos are going to be 30 frames in length
sequence_length = 60

In [21]:
# for person in persons:
#     for view in views:
#         for sequence in range(no_sequences):
#             try: 
#                 os.makedirs(os.path.join(DATA_PATH, person, view, str(sequence)))
#             except:
#                 pass

# Dataset Collection

In [22]:
# cap = cv2.VideoCapture(1)
# for person in persons:
#     for view in views:
#         for sequence in range(no_sequences):
#             for frame_num in range(sequence_length):

#                 ret, frame = cap.read()
                    
#                 # Resize image
#                 image = frame.copy()
#                 image = tf.image.resize_with_pad(tf.expand_dims(image, axis=0),192,256)
#                 input_img = tf.cast(image, dtype=tf.int32)
                    
#                 # Detection section
#                 results = movenet(input_img)
#                 extracted_keypoints = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
                    
#                 # Render keypoints 
#                 loop_through_people(frame, extracted_keypoints, EDGES, 0.35)

#                 if frame_num == 0: 
#                     cv2.putText(frame, 'Starting {} Collection'.format(person), (120,200), 
#                             cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 1, cv2.LINE_AA)
#                     cv2.putText(frame, 'Collecting frames for {} with {} view, Video Number {}'.format(person, view, sequence), (15,12), 
#                             cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
#                     cv2.waitKey(3000)
#                 else: 
#                     cv2.putText(frame, 'Capturing'.format(person), (120,200), 
#                             cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 1, cv2.LINE_AA)
#                     cv2.putText(frame, 'Collecting frames for {} with {} view, Video Number {}'.format(person, view, sequence), (15,12), 
#                             cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                        
#                 # NEW Export keypoints
#                 keypoints = extracted_keypoints
#                 npy_path = os.path.join(DATA_PATH, person, view, str(sequence), str(frame_num))
#                 np.save(npy_path, keypoints)
                    
#                 cv2.imshow('Human Gait Identification', frame)
        
#                 if cv2.waitKey(10) & 0xFF== 27:
#                     break
# cap.release()
# cv2.destroyAllWindows()

# Creating labels and Features

In [23]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [24]:
label_map = {label:num for num, label in enumerate(persons)}

In [25]:
label_map

{'Lance': 0, 'Seif': 1}

In [32]:
sequences, labels = [], []
for person in persons:
    for view in views:
        for sequence in np.array(os.listdir(os.path.join(DATA_PATH, person, view))).astype(int):
            window = []
            for frame_num in range(sequence_length):

                res = np.load(os.path.join(DATA_PATH, person, view, str(sequence), "{}.npy".format(frame_num)))
                first_person_data = res[0]
                window.append(first_person_data.flatten())
            sequences.append(window)
            labels.append(label_map[person])

In [33]:
np.array(sequences).shape

(160, 60, 51)

In [34]:
np.array(labels).shape

(160,)

In [35]:
X = np.array(sequences)

In [36]:
X.shape

(160, 60, 51)

In [37]:
y = to_categorical(labels).astype(int)

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [39]:
y_test.shape

(8, 2)

# CNN Model

In [40]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [41]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [43]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(60, 51)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(persons.shape[0], activation='softmax'))

In [44]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [45]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
5/5 [==============================] - 3s 66ms/step - loss: 0.6935 - categorical_accuracy: 0.5066
Epoch 2/2000
5/5 [==============================] - 0s 62ms/step - loss: 0.6975 - categorical_accuracy: 0.5066
Epoch 3/2000
5/5 [==============================] - 0s 63ms/step - loss: 0.6917 - categorical_accuracy: 0.5066
Epoch 4/2000
5/5 [==============================] - 0s 63ms/step - loss: 0.6899 - categorical_accuracy: 0.5000
Epoch 5/2000
5/5 [==============================] - 0s 63ms/step - loss: 0.6873 - categorical_accuracy: 0.5592
Epoch 6/2000
5/5 [==============================] - 0s 63ms/step - loss: 0.6704 - categorical_accuracy: 0.6053
Epoch 7/2000
5/5 [==============================] - 0s 64ms/step - loss: 0.7129 - categorical_accuracy: 0.5789
Epoch 8/2000
5/5 [==============================] - 0s 64ms/step - loss: 0.7008 - categorical_accuracy: 0.4803
Epoch 9/2000
5/5 [==============================] - 0s 64ms/step - loss: 0.6930 - categorical_accuracy: 0.5066
E

In [2]:
model.summary()

NameError: name 'model' is not defined

In [1]:
print("Evaluate on test data")
results = model.evaluate(X_test, y_test, batch_size=128)
print("test loss, test acc:", results)

Evaluate on test data


NameError: name 'model' is not defined

In [50]:
res = model.predict(X_test)
res[0]

1/1 [==============================] - 0s 24ms/step


array([0.00163027, 0.99836975], dtype=float32)

In [51]:
model.save('train1.h5')

In [52]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [53]:
yhat = model.predict(X_test)

1/1 [==============================] - 0s 17ms/step


In [54]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [55]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[4, 0],
        [0, 4]],

       [[4, 0],
        [0, 4]]], dtype=int64)

In [56]:
accuracy_score(ytrue, yhat)

1.0

In [57]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [58]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.35

cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    
    # Resize image
    img = frame.copy()
    img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0),192,256)
    input_img = tf.cast(img, dtype=tf.int32)
    
    # Detection section
    results = movenet(input_img)
    keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
    
    # Render keypoints 
    loop_through_people(frame, keypoints_with_scores, EDGES, threshold)

    sequence.append(keypoints_with_scores)
    sequence = sequence[-30:]
        
    if len(sequence) == 30:
        res = model.predict(np.expand_dims(sequence, axis=0))[0]
        print(persons[np.argmax(res)])

    cv2.imshow('Human Gait Identification', frame)
    
    if cv2.waitKey(10) & 0xFF== 27:
        break
cap.release()
cv2.destroyAllWindows()

InvalidArgumentError: Graph execution error:

Detected at node 'sequential_1/lstm_3/transpose' defined at (most recent call last):
    File "c:\Users\lance\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\lance\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\lance\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\lance\AppData\Roaming\Python\Python310\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "C:\Users\lance\AppData\Roaming\Python\Python310\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "C:\Users\lance\AppData\Roaming\Python\Python310\site-packages\tornado\platform\asyncio.py", line 205, in start
      se

: 